In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoTokenizer,AutoModelForCausalLM, AutoModel
import evaluate
import sys
sys.path.insert(0, '../lit-gpt/generate')
from pathlib import Path
import lora
import base
from datasets import load_dataset
import csv
import random
import torch
from tqdm import tqdm
import torch.nn as nn
import os
import json
import pandas as pd

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../data_final_split_995/test.json") as f:
    test_data = json.load(f)
with open("../data_final_split_995/train.json") as f:
    train_data = json.load(f)
with open("eval_dataset.json") as f:
    eval_data = json.load(f)
print("length train: "+str(len(train_data)))
print("length test: "+str(len(test_data)))
eval_data = list(eval_data.items())
print("length eval: "+str(len(eval_data)))

length train: 3035
length test: 159
length eval: 2


In [19]:
eval_data[0][1][2]

'How can I reach Aalen University'

In [3]:
state_dict = torch.load('../lit-gpt/out/converted/mistralv2.pth')
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", state_dict=state_dict)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", state_dict=state_dict)
device = torch.device('cuda:3')  # Use GPU if available, else CPU
def compute_perplexity(data,model):
    model = model.to(device) 
    encodings = tokenizer("\n\n".join(data), return_tensors="pt")
    max_length = model.config.max_position_embeddings
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        input_ids=input_ids.to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        target_ids = target_ids.to(device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids,labels=target_ids)
            neg_log_likelihood = outputs.loss
        nlls.append(neg_log_likelihood)
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).mean())
    ppl = ppl.item()
    ppl = round(ppl,2)
    return (ppl)

In [8]:

def train_eval(data_len,data,samples,file_name):    
    results =[]
    n = random.sample(range(data_len-1), samples)
    for i in range(samples-1):
        #../lit-gpt/out/lora/mistral/lit_model_lora_finetunedV4.pth"
        bleu = evaluate.load("bleu")
        rouge = evaluate.load("rouge")
        model_output = lora.main(prompt = data[n[i]][0], checkpoint_dir=Path("../lit-gpt/checkpoints/mistralai/Mistral-7B-v0.1"),lora_path=Path("../lit-gpt/out/lora/mistral/lit_model_lora_finetunedV4.pth"))
        #model_output = base.main(prompt = data[n[i]][0], checkpoint_dir=Path("../lit-gpt/out/lora/merged/v3_3k/"))
        bleu_result= bleu.compute(references=[data[n[i]][2]], predictions=[model_output])
        rogue_results= rouge.compute(references=[data[n[i]][2]], predictions=[model_output])
        ppl=compute_perplexity(data[n[i]][2],model)
        results.append([data[n[i]][0],model_output,data[n[i]][2],ppl,round(bleu_result["bleu"],2),round(rogue_results["rougeLsum"],2)])

    column_names = ["Input", "Output","Reference", "Perplexity", "BLEU", "Rogue"]
    # Write the data to a CSV file
    with open(file_name+'.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for row in results:
            writer.writerow(row)

def test_eval(data_len,data,file_name):    
    results =[]
    for i in range(data_len-1):
        bleu = evaluate.load("bleu")
        rouge = evaluate.load("rouge")
        model_output = lora.main(prompt = data[i][0], checkpoint_dir=Path("../lit-gpt/checkpoints/mistralai/Mistral-7B-v0.1"),lora_path=Path("../lit-gpt/out/lora/mistral/lit_model_lora_finetunedV4.pth"))

        bleu_result= bleu.compute(references=[data[i][2]], predictions=[model_output])
        rogue_results= rouge.compute(references=[data[i][2]], predictions=[model_output])
        ppl=compute_perplexity(data[i][2],model)
        results.append([data[i][0],model_output,data[i][2],ppl,round(bleu_result["bleu"],2),round(rogue_results["rougeLsum"],2)])

    column_names = ["Input", "Output","Reference", "Perplexity", "BLEU", "Rogue"]
    # Write the data to a CSV file
    with open(file_name+'.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for row in results:
            writer.writerow(row)

def eval_eval(data_len,data,file_name):
    results =[]
    column_names = ["Input", "Output","Reference", "Perplexity", "BLEU", "Rogue"]
    
    for i in range(9,data_len-1):
            bleu = evaluate.load("bleu")
            rouge = evaluate.load("rouge")
            print("...")
            model_output = lora.main(prompt = data[0][1][i], checkpoint_dir=Path("../lit-gpt/checkpoints/mistralai/Mistral-7B-v0.1"),lora_path=Path("../lit-gpt/out/lora/mistral/lit_model_lora_finetunedV3.pth"))
            ppl=compute_perplexity(data[1][1][i],model)
            bleu_result= bleu.compute(references=[data[1][1][i]], predictions=[model_output])
            rogue_results= rouge.compute(references=[data[1][1][i]], predictions=[model_output])

            results.append([data[0][1][i],model_output,data[1][1][i],ppl,round(bleu_result["bleu"],2),round(rogue_results["rougeLsum"],2)])

    with open(file_name+'.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for row in results:
            writer.writerow(row)

In [ ]:
train_eval(len(train_data),train_data,150,"train_v3_3k")
#test_eval(len(test_data),test_data,"test_")

In [9]:
eval_eval(16,eval_data,"eval_v3")

...


Loading model '../lit-gpt/checkpoints/mistralai/Mistral-7B-v0.1/lit_model.pth' with {'name': 'Mistral-7B-v0.1', 'hf_config': {'org': 'mistralai', 'name': 'Mistral-7B-v0.1'}, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 512, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 8, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 14336, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_token': 0, 'r': 8, 'alpha': 16, 'dropout': 0.05, 'to_query': True, 'to_key': True, 'to_value': True, 'to_projection': False, 'to_mlp': False, 'to_head': False, 'head_size': 128, 'rope_n_elem': 128}
Time to instantiate model: 0.07 seconds.
Time to load the model weights: 12.73 seconds.
Seed set to 1234


Time for inference: 1.79 sec total,

The campus has various student organizations or clubs, including the Fakultätsrat, Studienberatung, Sportwirtschaftliche Vereinigung, and Studienkolleg Konstanz.


  0%|          | 0/5 [00:00<?, ?it/s]


...


...

  0%|          | 0/4 [00:00<?, ?it/s]


...


Loading model '../lit-gpt/checkpoints/mistralai/Mistral-7B-v0.1/lit_model.pth' with {'name': 'Mistral-7B-v0.1', 'hf_config': {'org': 'mistralai', 'name': 'Mistral-7B-v0.1'}, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 512, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 8, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 14336, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_token': 0, 'r': 8, 'alpha': 16, 'dropout': 0.05, 'to_query': True, 'to_key': True, 'to_value': True, 'to_projection': False, 'to_mlp': False, 'to_head': False, 'head_size': 128, 'rope_n_elem': 128}
Time to instantiate model: 0.06 seconds.
Time to load the model weights: 12.85 seconds.
Seed set to 1234


Time for inference: 0.79 sec total,

Die Studiengebühr beträgt 186 Euro pro Semester.


  0%|          | 0/4 [00:00<?, ?it/s]


In [10]:
# Load the CSV data into a pandas DataFrame
test = pd.read_csv('test_v1_6k.csv')
train = pd.read_csv('train_v1_6k.csv')
eval = pd.read_csv('eval_.csv')

# Compute and print other statistical parameters
print("Train")
print(train.describe())
print("")
print("Test")
print(test.describe())
print("")
print("Eval")
print(eval.describe())

Train
       Perplexity        BLEU       Rogue
count  299.000000  299.000000  299.000000
mean     2.213110    0.623170    0.746629
std      0.380156    0.349394    0.269124
min      1.630000    0.000000    0.050000
25%      1.970000    0.341808    0.559900
50%      2.100000    0.678369    0.846154
75%      2.310000    1.000000    1.000000
max      3.960000    1.000000    1.000000

Test
       Perplexity        BLEU       Rogue
count  158.000000  158.000000  158.000000
mean     2.154367    0.539843    0.700639
std      0.326294    0.342981    0.263484
min      1.630000    0.000000    0.153846
25%      1.960000    0.240998    0.500000
50%      2.090000    0.515790    0.750927
75%      2.260000    0.862545    0.963268
max      3.450000    1.000000    1.000000

Eval
       Perplexity
count   49.000000
mean     2.827551
std      0.656460
min      1.980000
25%      2.240000
50%      2.860000
75%      3.310000
max      4.450000
